# Submit data to Reactome Analysis Service

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pylab as plt
import matplotlib
from IPython.display import display, HTML, Image

import numpy as np
import pandas as pd

%matplotlib inline

In [49]:
import requests 
from urllib.parse import quote
import json

Load test data

In [87]:
threshold = 0.05
colname = 'HK_vs_UN'
padj_colname = 'padj_HK_vs_UN'
fc_colname = 'FC_HK_vs_UN'
species = 'Mus musculus'

In [27]:
df = pd.read_pickle('C:\\Users\\joewa\\Work\\git\\GraphOmics\\graphomics\\static\\data\\debugging\\reactome_df.p')
df = df[df[padj_colname] > threshold]
df = df[fc_colname].to_frame()
df = np.power(df, 2)
df.head(10)

,FC_HK_vs_UN
ENSMUSG00000000001,0.017421
ENSMUSG00000000028,0.217985
ENSMUSG00000000049,1.695977
ENSMUSG00000000058,0.013804
ENSMUSG00000000085,0.003827
ENSMUSG00000000125,11.414734
ENSMUSG00000000126,0.004124
ENSMUSG00000000127,0.000495
ENSMUSG00000000149,0.300646
ENSMUSG00000000154,0.076876


In [52]:
df.head(33)

,FC_HK_vs_UN
ENSMUSG00000000001,0.017421
ENSMUSG00000000028,0.217985
ENSMUSG00000000049,1.695977
ENSMUSG00000000058,0.013804
ENSMUSG00000000085,0.003827
ENSMUSG00000000125,11.414734
ENSMUSG00000000126,0.004124
ENSMUSG00000000127,0.000495
ENSMUSG00000000149,0.300646
ENSMUSG00000000154,0.076876


In [54]:
data = df.to_csv(sep='\t', header=True, index_label='#id', float_format='%.15f')
data[0:100]

'#id\tFC_HK_vs_UN\r\nENSMUSG00000000001\t0.017420857886583\r\nENSMUSG00000000028\t0.217984642810172\r\nENSMUSG'

In [55]:
encoded_species = quote(species)
encoded_species

'Mus%20musculus'

In [56]:
url = 'https://reactome.org/AnalysisService/identifiers/?interactors=false&species=' + encoded_species + '&pageSize=20&page=1&sortBy=ENTITIES_PVALUE&order=ASC&resource=TOTAL&pValue=1&includeDisease=true'
url

'https://reactome.org/AnalysisService/identifiers/?interactors=false&species=Mus%20musculus&pageSize=20&page=1&sortBy=ENTITIES_PVALUE&order=ASC&resource=TOTAL&pValue=1&includeDisease=true'

In [57]:
response = requests.post(url, headers={'Content-Type': 'text/plain'}, data=data.encode('utf-8'))

In [64]:
response.status_code

200

In [58]:
json_response = json.loads(response.text)
json_response

{'summary': {'token': 'MjAyMDAzMDIxMTQwMjlfMzU2Mzc%3D',
  'projection': False,
  'interactors': False,
  'type': 'EXPRESSION',
  'sampleName': 'id',
  'text': True},
 'expression': {'columnNames': ['FC_HK_vs_UN'],
  'min': 0.0,
  'max': 11.414733636935884},
 'identifiersNotFound': 62,
 'pathwaysFound': 1186,
 'pathways': [{'stId': 'R-MMU-9006934',
   'dbId': 9778010,
   'name': 'Signaling by Receptor Tyrosine Kinases',
   'species': {'dbId': 48892, 'taxId': '10090', 'name': 'Mus musculus'},
   'llp': False,
   'entities': {'resource': 'TOTAL',
    'total': 463,
    'found': 53,
    'ratio': 0.0418625678119349,
    'pValue': 5.774137334935148e-07,
    'fdr': 0.0007702699204803487,
    'exp': [0.49574453417741493]},
   'reactions': {'resource': 'TOTAL',
    'total': 508,
    'found': 152,
    'ratio': 0.06543019062339},
   'inDisease': False},
  {'stId': 'R-MMU-69206',
   'dbId': 9777696,
   'name': 'G1/S Transition',
   'species': {'dbId': 48892, 'taxId': '10090', 'name': 'Mus musculus'

In [65]:
token = json_response['summary']['token']
token

'MjAyMDAzMDIxMTQwMjlfMzU2Mzc%3D'

In [84]:
pathways = json_response['pathways']

In [85]:
# https://stackoverflow.com/questions/6027558/flatten-nested-dictionaries-compressing-keys
pathways_df = pd.io.json.json_normalize(pathways, sep='_')

In [88]:
pathways_df = pathways_df[['stId', 'name', 'entities_fdr']].set_index('stId').rename(columns={
    'name': 'pw_name',
    'entities_fdr': 'REACTOME %s comb_p' % (colname)
})

In [89]:
pathways_df

,pw_name,REACTOME HK_vs_UN comb_p
stId,,
R-MMU-9006934,Signaling by Receptor Tyrosine Kinases,0.000770
R-MMU-69206,G1/S Transition,0.003153
R-MMU-453279,Mitotic G1 phase and G1/S transition,0.008330
R-MMU-68962,Activation of the pre-replicative complex,0.008330
R-MMU-1280215,Cytokine Signaling in Immune system,0.018554
R-MMU-352230,Amino acid transport across the plasma membrane,0.018554
R-MMU-5673000,RAF activation,0.030653
R-MMU-449147,Signaling by Interleukins,0.036756
R-MMU-69002,DNA Replication Pre-Initiation,0.040173
